Chameleon/FABRIC setup

In [1]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease as lease_manager
import chi.server as server_manager
import chi.network as network_manager

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

In [2]:
# Change to your project (CHI-XXXXXX) 
chi.set("project_name", "CH-822154")

chameleon_prefix =  f'{os.environ["USER"]}_fabric_iperf'
chameleon_server_name = chameleon_prefix+'_server'
chameleon_network_name = chameleon_prefix+'_net'
chameleon_subnet_name = chameleon_prefix+'_subnet'
chameleon_router_name = chameleon_prefix+'_router'
chameleon_lease_name = chameleon_prefix+'_lease'

chameleon_image_name='CC-Ubuntu20.04'

chameleon_site_1='CHI@TACC'
site_1_suffix='_tacc'
site_1_server_count=1

chameleon_site_2='CHI@UC'
site_2_suffix='_uc'
site_2_server_count=1

chameleon_key_name='chameleon_key_desktop'
chameleon_key_path='/home/mwhicks2_ncsu_edu/work/.ssh/my_chameleon_key.pem'

In [13]:
fablib = fablib_manager()
fablib.show_config()

fabric_slice_name='chameleon_stitch'

fabric_site_1='TACC'
site_1_slice_name="tacc_stitch"
site_1_facility_port='Chameleon-TACC'

fabric_site_2='STAR'
site_2_slice_name="chicago_stitch"
site_2_facility_port='Chameleon-StarLight'

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/mwhicks2_ncsu_edu/work/fabric_config/id_token.json
Project ID,1630021f-0a0c-4792-a241-997f410d36e1
Bastion Username,mwhicks2_0037146471
Bastion Private Key File,/home/mwhicks2_ncsu_edu/work/fabric_config/fabric_bastion_key_mobile
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/mwhicks2_ncsu_edu/work/fabric_config/slice_key.pub
Slice Private Key File,/home/mwhicks2_ncsu_edu/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Make Chameleon reservations

In [4]:
site_1_resource_lease_id='44b3f34b-cc36-42cd-a8d6-c886660856c4'
site_2_resource_lease_id=None

In [6]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

# create or get facilities lease
if site_1_resource_lease_id is None:
    # set start/end dates
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)
    
    # create lease reservations
    site_1_resource_reservations = []
    lease_manager.add_node_reservation(site_1_resource_reservations, count=site_1_server_count)
    lease_manager.add_fip_reservation(site_1_resource_reservations, count=site_1_server_count)
    
    # create lease
    site_1_resource_lease_name = chameleon_lease_name+'_resources'+site_1_suffix
    site_1_resource_lease = lease_manager.create_lease(site_1_resource_lease_name,
                                                      reservations=site_1_resource_reservations,
                                                      start_date=start_date,
                                                      end_date=end_date)
    site_1_resource_lease_id = lease_manager.get_lease_id(site_1_resource_lease_name)
# get lease info
site_1_node_reservation_id = lease_manager.get_node_reservation(site_1_resource_lease_id)

# set start/end dates
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)

# create network lease reservations
site_1_network_name = chameleon_network_name+site_1_suffix
site_1_network_reservations = []
site_1_network_reservations.append(
        {
            "resource_type": "network",
            "network_name": site_1_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# create network lease
site_1_network_lease_name = chameleon_lease_name+'_network'+site_1_suffix
site_1_network_lease = lease_manager.create_lease(site_1_network_lease_name,
                                                 reservations=site_1_network_reservations,
                                                 start_date=start_date,
                                                 end_date=end_date)
site_1_network_lease_id = lease_manager.get_lease_id(site_1_network_lease_name)

# get lease info
site_1_network_reservation_id = [reservation for reservation in site_1_network_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [7]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

# create or get facilities lease
if site_2_resource_lease_id is None:
    # reset start/end dates
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)
    
    # create lease reservations
    site_2_resource_reservations = []
    lease_manager.add_node_reservation(site_2_resource_reservations, count=site_2_server_count)
    lease_manager.add_fip_reservation(site_2_resource_reservations, count=site_2_server_count)
    
    # create lease
    site_2_resource_lease_name = chameleon_lease_name+'_resources'+site_2_suffix
    site_2_resource_lease = lease_manager.create_lease(site_2_resource_lease_name,
                                                      reservations=site_2_resource_reservations,
                                                      start_date=start_date,
                                                      end_date=end_date)
    site_2_resource_lease_id = lease_manager.get_lease_id(site_2_resource_lease_name)
# get lease info
site_2_node_reservation_id = lease_manager.get_node_reservation(site_2_resource_lease_id)

# set start/end dates
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)

# create network lease reservations
site_2_network_name = chameleon_network_name+site_2_suffix
site_2_network_reservations = []
site_2_network_reservations.append(
        {
            "resource_type": "network",
            "network_name": site_2_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# create network lease
site_2_network_lease_name = chameleon_lease_name+'_network'+site_2_suffix
site_2_network_lease = lease_manager.create_lease(site_2_network_lease_name,
                                                 reservations=site_2_network_reservations,
                                                 start_date=start_date,
                                                 end_date=end_date)
site_2_network_lease_id = lease_manager.get_lease_id(site_2_network_lease_name)

# get lease info
site_2_network_reservation_id = [reservation for reservation in site_2_network_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


Check networks

In [11]:
site_1_network_vlan = site_2_network_vlan = None
site_1_network_id = site_2_network_id = None

# uc check
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

#Get the network
site_1_network = network_manager.get_network(site_1_network_name)

#Get the network ID
site_1_network_id = site_1_network['id']

#Get the VLAN tag (needed for FABRIC stitching)
site_1_network_vlan = site_1_network['provider:segmentation_id']

# tacc check
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

#Get the network
site_2_network = network_manager.get_network(site_2_network_name)

#Get the network ID
site_2_network_id = site_2_network['id']

#Get the VLAN tag (needed for FABRIC stitching)
site_2_network_vlan = site_2_network['provider:segmentation_id']

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org
Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


FABRIC network slice

In [21]:
fabric_slice = fablib.new_slice(name=fabric_slice_name)

site_1_facility_port = fabric_slice.add_facility_port(name=site_1_facility_port, site=fabric_site_1, vlan=str(site_1_network_vlan))
site_1_facility_port_iface = site_1_facility_port.get_interfaces()[0]
site_2_facility_port = fabric_slice.add_facility_port(name=site_2_facility_port, site=fabric_site_2, vlan=str(site_2_network_vlan))
site_2_facility_port_iface = site_2_facility_port.get_interfaces()[0]

fabric_net = fabric_slice.add_l3network(name=f'facility_port_fabnetv4', interfaces=[site_1_facility_port_iface, site_2_facility_port_iface])

fabric_slice.submit()
fabric_slice = fablib.get_slice(name=fabric_slice_name)

In [16]:
fabric_network = fabric_slice.get_network('facility_port_fabnetv4')

subnet = fabric_network.get_subnet()
fabric_gateway_ip = fabric_network.get_gateway()

available_ips = list(subnet)[1:]

chameleon_gateway_ip=available_ips.pop(0)    
chameleon_allocation_pool_start=available_ips[0]
chameleon_allocation_pool_end=available_ips[10]

print(f'fabric_gateway_ip: {fabric_gateway_ip}')
print(f'chameleon_gateway_ip: {chameleon_gateway_ip}')
print(f'chameleon_allocation_pool_start: {chameleon_allocation_pool_start}')
print(f'chameleon_allocation_pool_end: {chameleon_allocation_pool_end}')

fabric_gateway_ip: facility_port_fabnetv4.gateway
chameleon_gateway_ip: a
chameleon_allocation_pool_start: c
chameleon_allocation_pool_end: t


In [19]:
print(str(subnet))

facility_port_fabnetv4.subnet


Add Chameleon subnets

In [17]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

site_1_subnet_name = chameleon_subnet_name+site_1_suffix

site_1_subnet = network_manager.create_subnet(site_1_subnet_name, site_1_network_id,
                                         cidr=str(subnet),
                                         allocation_pool_start=chameleon_allocation_pool_start,
                                         allocation_pool_end=chameleon_allocation_pool_end,
                                         gateway_ip=chameleon_gateway_ip)

chi.neutron().update_subnet(subnet=site_1_subnet['id'] ,
                                    body={
                                         "subnet": { 
                                             "host_routes": [ 
                                                    {
                                                        "destination": f"{fablib.FABNETV4_SUBNET}", 
                                                         "nexthop": f"{fabric_gateway_ip}"
                                                    }
                                             ] 
                                         }
                                    })

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


BadRequest: Invalid input for operation: facility_port_fabnetv4.subnet is not in a CIDR format.
Neutron server returns request_ids: ['req-95ee7f5a-c22c-4bef-9548-b9a105efd91b']

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

site_2_subnet_name = chameleon_subnet_name+site_2_suffix

site_2_subnet = network_manager.create_subnet(site_2_subnet_name, site_2_network_id,
                                         cidr=str(subnet),
                                         allocation_pool_start=chameleon_allocation_pool_start,
                                         allocation_pool_end=chameleon_allocation_pool_end,
                                         gateway_ip=chameleon_gateway_ip)

chi.neutron().update_subnet(subnet=site_2_subnet['id'] ,
                                    body={
                                         "subnet": { 
                                             "host_routes": [ 
                                                    {
                                                        "destination": f"{fablib.FABNETV4_SUBNET}", 
                                                         "nexthop": f"{fabric_gateway_ip}"
                                                    }
                                             ] 
                                         }
                                    })

Add Chameleon routers

In [ ]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

site_1_router_name = chameleon_router_name+site_1_suffix

site_1_router = network_manager.create_router(site_1_router_name, gw_network_name='public')
site_1_connection_port = network_manager.add_subnet_to_router_by_name(site_1_router_name, site_1_subnet_name)

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

site_2_router_name = chameleon_router_name+site_2_suffix

site_2_router = network_manager.create_router(site_2_router_name, gw_network_name='public')
site_2_connection_port = network_manager.add_subnet_to_router_by_name(site_2_router_name, site_2_subnet_name)

Start Chameleon servers

In [ ]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

site_1_servers = []
site_1_server_ids = []

site_1_server_name = chameleon_server_name+site_1_suffix

for i in range(site_1_server_count):
    server_name = f"{site_1_server_name}_{i+1}"
    
    site_1_servers.append(server_manager.create_server(server_name,
                                               reservation_id=site_1_node_reservation_id,
                                               network_name=site_1_network_name,
                                               image_name=chameleon_image_name,
                                               key_name=chameleon_key_name
                                            ))
    site_1_server_ids.append(server_manager.get_server_id(server_name))

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

site_2_servers = []
site_2_server_ids = []

site_2_server_name = chameleon_server_name+site_2_suffix

for i in range(site_2_server_count):
    server_name = f"{site_2_server_name}_{i+1}"
    
    site_2_servers.append(server_manager.create_server(server_name,
                                               reservation_id=site_2_node_reservation_id,
                                               network_name=site_2_network_name,
                                               image_name=chameleon_image_name,
                                               key_name=chameleon_key_name
                                            ))
    site_2_server_ids.append(server_manager.get_server_id(server_name))

Wait for servers

In [ ]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

for server in site_1_servers:
    server_manager.wait_for_active(server.id)
    
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

for server in site_2_servers:
    server_manager.wait_for_active(server.id)

Get server fixed IPs

In [ ]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

site_1_fixed_ips={}
for i in range(site_1_server_count):
    server_name = f"{site_1_server_name}_{i+1}"
    server_id = server_manager.get_server_id(server_name)
    fixed_ip = server_manager.get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    site_1_fixed_ips[server_name] = fixed_ip

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

site_2_fixed_ips={}
for i in range(site_2_server_count):
    server_name = f"{site_2_server_name}_{i+1}"
    server_id = serverr_manager.get_server_id(server_name)
    fixed_ip = server_manager.get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    site_2_fixed_ips[server_name] = fixed_ip

Run iPerf

In [ ]:
import paramiko
import random

node_1_addr=site_1_fixed_ips[f'{site_1_server_name}_{random.randint(1, site_1_server_count)}']
node_2_addr=site_2_fixed_ips[f'{site_2_server_name}_{random.randint(1, site_2_server_count)}']

key = paramiko.RSAKey.from_private_key_file(chameleon_key_path)
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(node_1_addr,username='cc',pkey = key)

stdin, stdout, stderr = client.exec_command('sudo apt -y install iperf3 2>&1 /dev/null && nohup iperf3 -s -1')
print (stdout.read())
print (stderr.read())

client.close()

key = paramiko.RSAKey.from_private_key_file(chameleon_key_path)
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(node_2_addr,username='cc',pkey = key)

stdin, stdout, stderr = client.exec_command(f'sudo apt -y install iperf3 2>&1 /dev/null && iperf3 -c {node_1_addr} -P 4 -t 30 -i 10 -O 10')
print (stdout.read())
print (stderr.read())

client.close()

free FABRIC resources

free Chameleon resources

In [ ]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

for server_id in site_1_server_ids:
    server_manager.delete_server(server_id)

In [22]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

network_manager.nuke_network(site_1_network_id)

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


Unauthorized: Unauthorized (HTTP 401)

In [ ]:
chi.use_site(chameleon_site_1)
os.environ['OS_REGION_NAME'] = chameleon_site_1

lease_manager.delete_lease(site_1_resource_lease_id)
lease_manager.delete_lease(site_1_network_lease_id)

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

for server_id in site_2_server_ids:
    server_manager.delete_server(server_id)

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

network_manager.nuke_network(site_2_network_id)

In [ ]:
chi.use_site(chameleon_site_2)
os.environ['OS_REGION_NAME'] = chameleon_site_2

lease_manager.delete_lease(site_2_resource_lease_id)
lease_manager.delete_lease(site_2_network_lease_id)